# First Tests and Ansatz

In [46]:
## Imports
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
import pandas as pd
import matplotlib.pyplot as plt

In [47]:
## Settings
tf.random.set_seed(42)
np.random.seed(42)

## 01 Running an easy 2 Body Problem in a gradient tape

In [48]:
@tf.function
def inverse_pairwise_distances(X: tf.Tensor) -> tf.Tensor:
    """
    Returns a matrix with the inverse paired euclidean distances of the instances (rows) in X. The diagonals get imputed with zeros.

    Parameters
    X : tf.Tensor of shape (N, 3)
        Tensor with instances to compue the pairwise distances between each other. Gets casted to float32.
   
    Returns
    tf.Tensor of shape (N, N)
        Pairwise distances in the upper or lower triangular part.
    """
    X = tf.cast(X, tf.float32)
    r = tf.reduce_sum(X * X, 1)
    r = tf.reshape(r, [-1, 1])
    D = r - 2.*tf.matmul(X, tf.transpose(X)) + tf.transpose(r)
    D = tf.linalg.set_diag(D, tf.repeat(np.inf, X.shape[0]))
    D = tf.linalg.band_part(1./D, 0, -1)
    return D

@tf.function
def pairwise_mass_product(M) -> tf.Tensor:
    Msq = tf.expand_dims(M, axis=0) * tf.expand_dims(M, axis=1)
    return Msq

@tf.function
def compute_potential(X: tf.Tensor, M: tf.Tensor) -> tf.Tensor:
    D = inverse_pairwise_distances(X)
    Msq = pairwise_mass_product(M)
    return tf.reduce_sum(D*Msq)
    
@tf.function
def compute_accelerations(grad: tf.Tensor, M: tf.Tensor) -> tf.Tensor:
    return grad / tf.reshape(M, (-1, 1))

In [49]:
X0 = tf.Variable([[0., 0., 0.], [1., 1., 0.], [2., 1., 0.]])
M0 = tf.constant([3., 1., 1.])

In [50]:
inverse_pairwise_distances(X0)

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[0. , 0.5, 0.2],
       [0. , 0. , 1. ],
       [0. , 0. , 0. ]], dtype=float32)>

In [51]:
compute_potential(X0, M0)

<tf.Tensor: shape=(), dtype=float32, numpy=3.1>

In [52]:
with tf.GradientTape() as tape:
    tape.watch(X0)
    pot_energy = compute_potential(X0, M0)
grad = tape.gradient(pot_energy, X0)
print(grad)
print(compute_accelerations(grad, M0))

tf.Tensor(
[[ 1.98        1.74        0.        ]
 [ 0.5        -1.5         0.        ]
 [-2.48       -0.24000001  0.        ]], shape=(3, 3), dtype=float32)
tf.Tensor(
[[ 0.66        0.58        0.        ]
 [ 0.5        -1.5         0.        ]
 [-2.48       -0.24000001  0.        ]], shape=(3, 3), dtype=float32)


## Runge Kutta Fehlberg integrator for the equations of Motion

The Runge-Kutte-Fehlberg Method is a numerical Method for integrating ordinary differential equations. The following c++ implementation is meant to integrate equations of motion with the acceleration `f`.

c++ Implementation:
```c++
void solver::runge_kutta_fehlberg2(vec3D& x, vec3D& v, double dt, std::function<vec3D(vec3D)> f) {
    vec3D k1v = f(x);
    vec3D k1x = v;
    vec3D k2v = f(x + dt * k1x / 4.);
    vec3D k2x = v + dt * k1v / 4.;
    vec3D k3v = f(x + dt * k1x * 3. / 32. + dt * k2x * 9. / 32.);
    vec3D k3x = v + dt * k1v * 3. / 32. + dt * k2v * 9. / 32.;
    vec3D k4v = f(x + dt * k1x * 1932. / 2197. - dt * k2x * 7200. / 2197. + dt * k3x * 7296. / 2197.);
    vec3D k4x = v + dt * k1v * 1932. / 2197. - dt * k2v * 7200. / 2197. + dt * k3v * 7296. / 2197.;
    vec3D k5v = f(x + dt * k1x * 439. / 216. - dt * k2x * 8. + dt * k3x * 3680. / 513 - dt * k4x * 845. / 4104.);
    vec3D k5x = v + dt * k1v * 439. / 216. - dt * k2v * 8. + dt * k3v * 3680. / 513 - dt * k4v * 845. / 4104.;
    vec3D k6v = f(x - dt * k1x * 8. / 27. + dt * k2x * 2. - dt * k3x * 3544. / 2565. + dt * k4x * 1859. / 4104. - dt * k5x * 11. / 40.);
    vec3D k6x = v - dt * k1v * 8. / 27. + dt * k2v * 2. - dt * k3v * 3544. / 2565. + dt * k4v * 1859. / 4104. - dt * k5v * 11. / 40.;
    x = x + dt * (16. / 135. * k1x + 6656. / 12825. * k3x + 28561. / 56430. * k4x - 9. / 50. * k5x + 2. / 55. * k6x);
    v = v + dt * (16. / 135. * k1v + 6656. / 12825. * k3v + 28561. / 56430. * k4v - 9. / 50. * k5v + 2. / 55. * k6v);
}
```